## Búsqueda de features ##


In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn import preprocessing

import xgboost as xgb
from sklearn import preprocessing

tf = pd.read_csv('../data/events_up_to_01062018.csv')
labels = pd.read_csv('../data/labels_training_set.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Divido las fechas**

In [3]:
tf["timestamp"] = pd.to_datetime(tf["timestamp"])
tf["month"] = tf["timestamp"].dt.month
tf["day"] = tf["timestamp"].dt.day
tf["hour"] = tf["timestamp"].dt.hour
tf['dayOfTheWeek'] = tf['timestamp'].dt.dayofweek

In [4]:
labels.count() #cantidad de labels que tengo

person    19414
label     19414
dtype: int64

**Divido por evento me quedo con las columnas obligatorias por evento**

En en análisis exporatorio me había dado cuenta que no todos los eventos tienen la misma información, por lo tanto, intentaré combinar los eventos con el objetivo de encontrar **nuevos features**. 

Para comenzar divido en distintos DataFrames los distintos eventos, eliminando información que no es útil. 

In [5]:
viewedProduct  = tf[tf['event'] == 'viewed product'].loc[:,['dayOfTheWeek','hour','month','day','person','sku','model','condition','storage','color']]
brandListing = tf[tf['event'] == 'brand listing'].loc[:,['dayOfTheWeek','hour','month','day','person','skus']]
visitedSite = tf[tf['event'] == 'visited site'].loc[:,['dayOfTheWeek','hour','month','day','person','channel','new_vs_returning','city','region','country','device_type',\
                                                       'screen resolution','operating system version','browser version']]
adCampaignHit = tf[tf['event'] == 'ad campaign hit'].loc[:,['dayOfTheWeek','hour','month','day','person','url','campaign_source']]
genericListing = tf[tf['event'] == 'generic listing'].loc[:,['dayOfTheWeek','hour','month','day','person','skus']]
searchedProduct = tf[tf['event'] == 'searched products'].loc[:,['dayOfTheWeek','hour','month','day','person','skus','search_term']]
searchEngineHit = tf[tf['event'] == 'search engine hit'].loc[:,['dayOfTheWeek','hour','month','day','person','search engine']]
checkout = tf[tf['event'] == 'checkout'].loc[:,['dayOfTheWeek','hour','month','day','person','sku','color','storage','model','condition']]
staticPage = tf[tf['event'] == 'staticpage'].loc[:,['dayOfTheWeek','hour','month','day','person','static page']]
conversion = tf[tf['event'] == 'conversion'].loc[:,['dayOfTheWeek','hour','month','day','person','sku','model','color','condition','storage']]
lead = tf[tf['event'] == 'lead'].loc[:,['dayOfTheWeek','hour','month','day','person','model']]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


**Buscar atributos de otro evento que no sea conversion**


La primera instancia que analizaré es si el usuario es nuevo o no, analizando del visited site el atributo 'new_vs_returning'

### 1º El usuario realiza un compra habiendo visitado la página una vez ###

Intentaré determinar ahora si el usuario que realizó una conversión sólo realizó una única visita al sitio. Para ello debo fijarme en visited site si existe algún valor que sea returning, en tal caso, deja de ser nuevo. 

In [6]:
newVsReturning = visitedSite.loc[:,['person','new_vs_returning']]
newVsReturning = newVsReturning.groupby('person').new_vs_returning.value_counts().to_frame('cantidad')
newVsReturning = newVsReturning.unstack().fillna(0)
newVsReturning = newVsReturning.reset_index()
newVsReturning.columns = ['person','new','returning']
newVsReturning['esNuevo'] = newVsReturning['returning'] == 0

Agrego este atributo a conversion con un merge

In [7]:
labels_f = pd.merge(labels, newVsReturning.loc[:,['esNuevo','person']], how = 'left', on = 'person')

In [8]:
labels_f['esNuevo'].isnull().value_counts()

False    19126
True       288
Name: esNuevo, dtype: int64

### 2º Cantidad de visitas del usuario a la página ###

Es decir que tengo 288 labels en los que no se si es nuevo o si ya volvió

Analizaré ahora la frecuencia de entradas al sitio de un determinado usuario. Para ello analizaré la relación entre new y returnings (le sumo 1 a la cantidad de returnings). 

In [9]:
newVsReturning['cantidadDeVisitas'] = (newVsReturning['new'] + newVsReturning['returning']).astype('float32')
labels_f = pd.merge(labels_f, newVsReturning.loc[:,['cantidadDeVisitas','person']],how = 'left', on = 'person')

In [10]:
labels_f['cantidadDeVisitas'].isnull().value_counts()

False    19126
True       288
Name: cantidadDeVisitas, dtype: int64

In [11]:
labels_f['cantidadDeVisitas'] = labels_f['cantidadDeVisitas'].astype('float32')
labels_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 4 columns):
person               19414 non-null object
label                19414 non-null int64
esNuevo              19126 non-null object
cantidadDeVisitas    19126 non-null float32
dtypes: float32(1), int64(1), object(2)
memory usage: 682.5+ KB


Me vuelve a dar lo mismo que antes, los resultados son consistentes. 

**Por lo tanto, el atributo esNuevo y cantidadDeVisitas no pueden aplicarse a 288 labels de 19414**

### 3º El usuario ingresó a la página por algún aviso publicitario alguna vez ###

Analizaré si el usario ingresó a la página desde una publicidad (en algún momento) para ello usaré el evento ad_campaing_hit.

In [12]:
personasAdHit = adCampaignHit.loc[:,['person']]
personasAdHit['ingresoPorPublicidad'] = True
personasAdHit = personasAdHit.drop_duplicates()

In [13]:
labels_f = pd.merge(labels_f, personasAdHit, on = 'person', how = 'left')
labels_f = labels_f.fillna(False)

In [14]:
labels_f['ingresoPorPublicidad'].isnull().value_counts()

False    19414
Name: ingresoPorPublicidad, dtype: int64

Supongo que la información que no tengo en ad_campaing_hit de los usuarios, es porque no ingresaron por una campaña publicitaria alguna vez a la página y por lo tanto **este atributo puede usarse para lo 19414 labels**. 

### 4º Campaña publicitaria por la que ingresó el usuario ###

Analizaré por qué campaña ingresaron los usuarios (en algún momento y luego realizaron una conversión), agregando esa información de ad_campaing_hit

In [15]:
campaniaPersonasAdHit = adCampaignHit.loc[:,['person','campaign_source']]
campaniaPersonasAdHit = campaniaPersonasAdHit.groupby('person').campaign_source.value_counts().to_frame('cantidad')
campaniaPersonasAdHit = campaniaPersonasAdHit.unstack().fillna(0).reset_index()
campaniaPersonasAdHit.head()
campaniaPersonasAdHit.columns=['person','cant_facebook','cant_facebookAds', 'cant_facebookSocial', 'cant_marketing_social', 'cant_afiliado', 'cant_afilio', 'cant_bing', 'cant_blog','cant_buscape', 'cant_criteo', 'cant_datacrush', 'cant_emblue', 'cant_gizmodo', 'cant_google', 'cant_indexa', 'cant_manifest', 'cant_mercadopago', 'cant_onsite', 'cant_rakuten','cant_rtbhouse', 'cant_voxus', 'cant_yotpo', 'cant_zanox']

**FALTA COMPLETAR ESTO, SON MUCHAS CAMPAÑAS**

### 5º información sobre el dispositivo por el cual se conectaron ###

El único evento que posee información sobre el dispositivo al cual se conecto es Visited Site. Se registra un Visited Site cada vez que el usuario ingresa a Trocafone (cualquiera sea el motivo). Repetiré el análisis hecho en el análisis exploratorio en el que mostrábamos que la mayoría de los usuarios se conectaban por un sólo dispositivo.


In [16]:
def contar_columnas(x):#(col1,col2,col3,col4):
    t=0
    if x['tablet']>0:
        t += 1
    if x['smartphone']>0:
        t += 1
    if x['computer']>0:
        t += 1
    if x['unknown']>0:
        t += 1
    return t


#print(visited_site.device_type.value_counts()) -> LOS UNKNOWN SON DESPRECIABLES. 
dispositivos= visitedSite.groupby('person')['device_type'].value_counts().to_frame('cantidad')
dispositivos = dispositivos.unstack().fillna(0).reset_index()
dispositivos.columns = ['person','computer','smartphone','tablet','unknown']
dispositivos = dispositivos.set_index('person')
dispositivos['porcentaje'] = dispositivos.apply(lambda p:contar_columnas(p), axis = 1)


print(dispositivos['porcentaje'].value_counts(normalize = True).to_frame())

#quito aquellos que se hayan conectado por más de un dispositivo
dispositivos = dispositivos[dispositivos['porcentaje'] <= 1]
visited_device = visitedSite.loc[:,['person','device_type']].drop_duplicates()
dispositivos = dispositivos.reset_index()
visited_device = pd.merge(dispositivos.loc[:,['person']],visited_device, on= 'person',how = 'left').drop_duplicates()


   porcentaje
1    0.959547
2    0.039721
3    0.000732


Por lo tanto existe un 96 porciento de los usuarios que se conectaron de un único tipo de dispositivo, por lo tanto, despreciaremos el otro 4 porciento. 


In [17]:
labels_f = pd.merge(labels_f, visited_device, how = 'left', on = 'person')
#le cambio el nombre a la columna device type para que no se confunda con lo que compraron

In [18]:
labels_f.device_type.isnull().value_counts()

False    18370
True      1044
Name: device_type, dtype: int64

In [19]:
labels_f['deviceSmartphone'] = labels_f['device_type'] == 'smartphone'
labels_f['deviceComputer'] = labels_f['device_type'] == 'computer'
labels_f['deviceTablet'] = labels_f['device_type'] == 'tablet'
labels_f['deviceUnknown'] = labels_f['device_type'] == 'unknown'

Es decir que **de los 19414 labels el atributo device_type presenta información para 18370** entonces 1044 labels no aportan este tipo de información. 

### 6º Cantidad de veces que el usuario vio un producto ###

Analizaré ahora la cantidad de veces que el usuario vio un producto 

In [20]:
cantidadDeVisitasAProductos = viewedProduct.loc[:,['person']]
cantidadDeVisitasAProductos['cantVisitasAVP'] = 1
cantidadDeVisitasAProductos = cantidadDeVisitasAProductos.groupby('person').cantVisitasAVP.sum().to_frame()


In [21]:
labels_f = pd.merge(labels_f, cantidadDeVisitasAProductos, on= 'person', how = 'left')

In [22]:
labels_f.cantVisitasAVP.isnull().value_counts()

False    18570
True       844
Name: cantVisitasAVP, dtype: int64

Supongo que como no tenemos información de 844 labels sobre la cantidad de visitas a productos estos nunca lo hicieron.

In [23]:
personasSinVisitas = labels_f.loc[:,['person','cantVisitasAVP']]
personasSinVisitas = personasSinVisitas.fillna(0)

In [24]:
labels_f = pd.merge(labels_f, personasSinVisitas, how = 'left', on = 'person')

In [25]:
labels_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 12 columns):
person                  19414 non-null object
label                   19414 non-null int64
esNuevo                 19414 non-null bool
cantidadDeVisitas       19414 non-null object
ingresoPorPublicidad    19414 non-null bool
device_type             18370 non-null object
deviceSmartphone        19414 non-null bool
deviceComputer          19414 non-null bool
deviceTablet            19414 non-null bool
deviceUnknown           19414 non-null bool
cantVisitasAVP_x        18570 non-null float64
cantVisitasAVP_y        19414 non-null float64
dtypes: bool(6), float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [26]:
labels_f = labels_f.loc[:,['person','label','esNuevo','cantidadDeVisitas','ingresoPorPublicidad','deviceSmartphone','deviceComputer','deviceTablet','deviceUnknown','cantVisitasAVP_y']]
labels_f.columns = ['person','label','esNuevo','cantidadDeVisitas','ingresoPorPublicidad','deviceSmartphone','deviceComputer','deviceTablet','deviceUnknown','cantVisitasAVP']

### 7º Momento del día en el que realizan las visitas ###

Usando visitedSite analizaré el momento del día en el que el usuario realizó las visitas. Lo agruparé en las siguientes clases:

-Mañana: si el usuario realizó la visita entre las 7 y 12 hs. 

-Tarde: si el usuario realizó la visita entre las 12 y las 18 hs. 

-Noche: si el usuario realizó la visita entre las 18 y 24 hs. 

-Madrugada: si el usuario realizó la visita entre las 24 y 7 hs. 

In [27]:
momentoDeVisita = visitedSite.loc[:,['person','hour']]
momentoDeVisita['maniana'] = ((momentoDeVisita['hour'] < 12 ) & (momentoDeVisita['hour'] >= 7)).astype('int32')
momentoDeVisita['tarde'] = ((momentoDeVisita['hour'] < 18 ) & (momentoDeVisita['hour'] >= 12)).astype('int32')
momentoDeVisita['noche'] = ((momentoDeVisita['hour'] < 24 ) & (momentoDeVisita['hour'] >= 18)).astype('int32')
momentoDeVisita['madrugada'] = ((momentoDeVisita['hour'] < 7 ) & (momentoDeVisita['hour'] >= 24)).astype('int32')
momentoDeVisita = momentoDeVisita.groupby('person').sum().reset_index()
momentoDeVisita = momentoDeVisita.loc[:,['person','maniana','tarde','noche','madrugada']]

In [28]:
labels_f = pd.merge(labels_f,momentoDeVisita, on = 'person', how = 'left') 

In [29]:
labels_f['maniana'].isnull().value_counts()

False    19126
True       288
Name: maniana, dtype: int64

Por lo tanto, **los atributos: maniana, tarde, noche y madrugada no fueron asignados para 288 labels**. Esto concuerda con el atributo device type obtenido anterior mentde del mismo evento: visited site.

### 8º Cantidad de visitas por canal que realiza el usuario###

Analizaré el canal por el que el usuario realiza la visita

In [30]:
canalDeVisita = visitedSite.loc[:,['person','channel']]
canalDeVisita = canalDeVisita.groupby('person').channel.value_counts().to_frame('cantidad')
canalDeVisita = canalDeVisita.unstack()
canalDeVisita = canalDeVisita.reset_index()
canalDeVisita.columns = ['person','cantDirect','cantEmail','cantOrganic','cantPaid','cantReferral',\
                           'cantSocial','cantUnknown']
canalDeVisita = canalDeVisita.fillna(0)

In [31]:
labels_f = pd.merge(labels_f,canalDeVisita,on = 'person', how = 'left')

In [32]:
labels_f['cantDirect'].isnull().value_counts()

False    19126
True       288
Name: cantDirect, dtype: int64

In [33]:
labels_f.columns

Index(['person', 'label', 'esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'deviceSmartphone', 'deviceComputer',
       'deviceTablet', 'deviceUnknown', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown'],
      dtype='object')

Por lo tanto, **los atributos: cantDirect, cantEmail, cantOrganic, cantPaid, cant referral, cantSocial, cantUnknown que refieren a la cantidad de accesos hecho por cada canal, no tiene fueron asignados para 288 labels**. Esto concuerda con todos los atributos obteneidos del mismo evento: vistedSite

### 9º Cantidad de visitas por día de la semana ###

Analizaré la cantidad de visitas que el usuario realiza por día de la semana. Usando el evento visited site. 

In [34]:
diaSemana = visitedSite.loc[:,['person','dayOfTheWeek']]
diaSemana = diaSemana.groupby('person').dayOfTheWeek.value_counts().to_frame('cantidad').unstack()

In [35]:
diaSemana.head()

cantidad                               
dayOfTheWeek        0    1    2    3    4    5     6
person                                              
0008ed71          NaN  NaN  NaN  2.0  NaN  NaN   NaN
00091926          2.0  4.0  3.0  6.0  3.0  5.0  11.0
00091a7a          1.0  NaN  NaN  NaN  NaN  NaN   NaN
000ba417          NaN  NaN  NaN  4.0  NaN  2.0   NaN
000c79fe          NaN  1.0  NaN  NaN  NaN  NaN   NaN

Supongo que los usuarios que no tiene información ese día de la semana es pq no visitaron la página en ese día de la semana. 

In [36]:
diaSemana = diaSemana.fillna(0)

In [37]:
diaSemana = diaSemana.reset_index()

In [38]:
diaSemana.columns = ['person','cantVisitasDomingo','cantVisitasLunes',\
                    'cantVisitasMartes', 'cantVisitasMiercoles', \
                    'cantVisitasJueves', 'cantVisitasViernes', \
                    'cantVisitasSabado']

In [39]:
labels_f = pd.merge(labels_f,diaSemana, how = 'left', on = 'person')

In [40]:
labels_f['cantVisitasMartes'].isnull().value_counts() 

False    19126
True       288
Name: cantVisitasMartes, dtype: int64

El 288 concuerda con los faltantes en visited site. 

### 10º cantidad de eventos que tiene registrado cada usuario ###

In [41]:
cantidadDeEventos = tf.loc[:,['event','person']]
cantidadDeEventos = cantidadDeEventos.loc[cantidadDeEventos['event'] != 'conversion']
#no tiene sentido este
cantidadDeEventos = cantidadDeEventos.groupby('person').event.count().to_frame()
cantidadDeEventos = cantidadDeEventos.reset_index()
cantidadDeEventos.columns = ['person','cantidadDeEventos']

In [42]:
labels_f = pd.merge(labels_f, cantidadDeEventos, on = 'person', how = 'inner' )


### 11º Cantidad de eventos, discretizado por tipo de evento, que tiene cada usuario ###

In [43]:
cantidadDeEventos = tf.loc[:,['event','person']]
cantidadDeEventos = cantidadDeEventos.loc[cantidadDeEventos['event'] != 'conversion']
#no tiene sentido este
cantidadDeEventos = cantidadDeEventos.groupby('person').event.value_counts().to_frame().unstack()
cantidadDeEventos = cantidadDeEventos.reset_index()
cantidadDeEventos.columns = ['person','cantAdCampaignHit', 'cantBandListing',\
                             'cantCheckout','cantGenericListing','cantLead',\
                            'cantSearchEngineHit', 'cantSearchedProducts',\
                             'cantStaticpage', 'cantViewedProduct', 'cantVisitedSite']

In [44]:
cantidadDeEventos.head()

,person,cantAdCampaignHit,cantBandListing,cantCheckout,cantGenericListing,cantLead,cantSearchEngineHit,cantSearchedProducts,cantStaticpage,cantViewedProduct,cantVisitedSite
0,0008ed71,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0
1,00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,372.0,34.0
2,00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
3,000ba417,1.0,24.0,6.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
4,000c79fe,1.0,NaN,1.0,1.0,NaN,1.0,9.0,NaN,3.0,1.0


Supongo que las personas que tiene NaN en un determinado evento es porque nunca lo realizaron, por lo tanto, lo completo con 0. 

In [45]:
cantidadDeEventos = cantidadDeEventos.fillna(0)

In [46]:
labels_f = pd.merge(labels_f,cantidadDeEventos,on = 'person', how = 'left')

In [47]:
labels_f.head()

,person,label,esNuevo,cantidadDeVisitas,ingresoPorPublicidad,deviceSmartphone,deviceComputer,deviceTablet,deviceUnknown,cantVisitasAVP,...,cantAdCampaignHit,cantBandListing,cantCheckout,cantGenericListing,cantLead,cantSearchEngineHit,cantSearchedProducts,cantStaticpage,cantViewedProduct,cantVisitedSite
0,0566e9c1,0,False,17,True,False,False,False,False,23.0,...,6.0,3.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,17.0
1,6ec7ee77,0,True,1,True,False,False,False,False,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,abe7a2fb,0,False,22,True,False,False,False,False,31.0,...,9.0,14.0,1.0,9.0,0.0,4.0,6.0,0.0,31.0,22.0
3,34728364,0,False,4,False,False,False,False,False,24.0,...,0.0,5.0,0.0,3.0,0.0,1.0,0.0,0.0,24.0,4.0
4,87ed62de,0,True,1,True,False,False,False,False,9.0,...,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0


### Agrego features según lo anterior pero en vez de la cantidad indico si lo hizo o no ###

### 12º El usuario vio un producto ### 

In [48]:
visitoUnProducto = viewedProduct.loc[:,['person']]
visitoUnProducto['cantVisitasAVP'] = 1
visitoUnProducto = visitoUnProducto.groupby('person').cantVisitasAVP.sum().to_frame()

In [49]:
visitoUnProducto['visitoUnProducto'] = visitoUnProducto.cantVisitasAVP > 0
visitoUnProducto = visitoUnProducto.reset_index()
visitoUnProducto = visitoUnProducto.loc[:,['person','visitoUnProducto']]

In [50]:
labels_f = pd.merge(labels_f,visitoUnProducto, on = 'person', how = 'left')

In [51]:
labels_f.visitoUnProducto.value_counts()

True    18570
Name: visitoUnProducto, dtype: int64

**Como todos los usuarios visitaron un producto este feature no sirve.**

### 13º El usuario visita la página en un momento determinado del día ###

In [52]:
momentoDeVisita_2 = momentoDeVisita
momentoDeVisita_2['visitaALaManiana'] = momentoDeVisita['maniana']>0
momentoDeVisita_2['visitaALaTarde'] = momentoDeVisita['tarde']>0
momentoDeVisita_2['visitaALaNoche'] = momentoDeVisita['noche']>0
momentoDeVisita_2['visitaALaMadrugada'] = momentoDeVisita['madrugada']>0
momentoDeVisita_2 = momentoDeVisita_2.loc[:,['person','visitaALaMañana','visitaALaNoche',\
                                         'visitaALaTarde', 'visitaALaMadrugada']]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Supongo que aquellos usuarios que no tienen información es porque no visitaron en ese momento. 

In [53]:
momentoDeVisita_2 = momentoDeVisita_2.fillna(False)

In [54]:
labels_f = pd.merge(labels_f, momentoDeVisita_2, on = 'person', how = 'inner')

### 14º el usuario visitó la página por determinado canal ###

In [55]:
canalDeVisita_2 = canalDeVisita
canalDeVisita_2['canalDirect'] = canalDeVisita['cantDirect'] >0 
canalDeVisita_2['canalEmail'] = canalDeVisita['cantEmail'] >0 
canalDeVisita_2['canalOrganic'] = canalDeVisita['cantOrganic'] >0 
canalDeVisita_2['canalPaid'] = canalDeVisita['cantPaid'] >0 
canalDeVisita_2['canalRefferal'] = canalDeVisita['cantReferral'] >0 
canalDeVisita_2['canalSocial'] = canalDeVisita['cantUnknown'] > 0

canalDeVisita_2 = canalDeVisita_2.loc[:,['person','canalDirect', 'canalEmail','canalOrganic',\
                                    'canalPaid','canalRefferal', 'canalSocial']]

In [56]:
labels_f = pd.merge(labels_f, canalDeVisita_2, on = 'person', how = 'left')

### 15º el usuario realizó determinado evento### 

In [57]:
cantidadDeEventos_2 = cantidadDeEventos
cantidadDeEventos_2['adCampaignHit'] = cantidadDeEventos['cantAdCampaignHit']>0
cantidadDeEventos_2['bandListing'] = cantidadDeEventos['cantBandListing']>0
cantidadDeEventos_2['checkout'] = cantidadDeEventos['cantCheckout']>0
cantidadDeEventos_2['genericListing'] = cantidadDeEventos['cantGenericListing']>0
cantidadDeEventos_2['searchEngineHit'] = cantidadDeEventos['cantSearchEngineHit']>0
cantidadDeEventos_2['searchedProducts'] = cantidadDeEventos['cantSearchedProducts']>0
cantidadDeEventos_2['staticpage'] = cantidadDeEventos['cantStaticpage']>0
cantidadDeEventos_2['viewedProduct'] = cantidadDeEventos['cantViewedProduct']>0
cantidadDeEventos_2['visitedSite'] = cantidadDeEventos['cantVisitedSite']>0


In [58]:
cantidadDeEventos_2 = cantidadDeEventos_2.loc[:,['person','adCampaignHit','bandListing','checkout','genericListing','searchEngineHit','searchedProducts','staticpage','viewedProduct','visitedSite']]

In [59]:
labels_f = pd.merge(labels_f,cantidadDeEventos_2, on = 'person', how = 'left')

### ANÁLISIS DE LOS ELEMENTOS NULL (FALTA HACER POR AHORA DROPPEO) ###

In [60]:
labels_f = labels_f.dropna()

# Análisis de la importancia de los features #
Lo que haré ahora es utilizar distintos features y ver cuales de ellos me aportan más o menos información.

## 1º Caso: todos los features ##

### Random Forest ###

In [61]:
labels_f['cantidadDeVisitas'] = labels_f['cantidadDeVisitas'].astype('float64')

In [160]:
labels_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18468 entries, 0 to 19125
Data columns (total 59 columns):
person                  18468 non-null object
label                   18468 non-null int64
esNuevo                 18468 non-null bool
cantidadDeVisitas       18468 non-null float64
ingresoPorPublicidad    18468 non-null bool
deviceSmartphone        18468 non-null bool
deviceComputer          18468 non-null bool
deviceTablet            18468 non-null bool
deviceUnknown           18468 non-null bool
cantVisitasAVP          18468 non-null float64
maniana                 18468 non-null float64
tarde                   18468 non-null float64
noche                   18468 non-null float64
madrugada               18468 non-null float64
cantDirect              18468 non-null float64
cantEmail               18468 non-null float64
cantOrganic             18468 non-null float64
cantPaid                18468 non-null float64
cantReferral            18468 non-null float64
cantSocial         

In [151]:
X = labels_f.loc[:,['esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown',
       'cantVisitasDomingo', 'cantVisitasLunes', 'cantVisitasMartes',
       'cantVisitasMiercoles', 'cantVisitasJueves', 'cantVisitasViernes',
       'cantVisitasSabado', 'cantidadDeEventos', 'cantAdCampaignHit',
       'cantBandListing', 'cantCheckout', 'cantGenericListing', 'cantLead',
       'cantSearchEngineHit', 'cantSearchedProducts', 'cantStaticpage',
       'cantViewedProduct', 'cantVisitedSite',
       'visitaALaMañana', 'visitaALaNoche', 'visitaALaTarde',
       'visitaALaMadrugada', 'adCampaignHit',
       'bandListing', 'genericListing', 'searchEngineHit',
       'searchedProducts', 'staticpage', 'viewedProduct', 'visitedSite']]



y =labels_f.loc[:,'label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

In [171]:

clf = RandomForestClassifier(bootstrap= 0.20, max_features= 45,n_jobs= 1500)
clf.fit(X_train,y_train)

#pred = clf.predict(X_test)
#print(accuracy_score(y_test, pred))
preds = clf.predict(X_test)
train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

RuntimeError: can't start new thread

### XGBOOST ###

In [153]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1350)

In [154]:
xg_reg = xgb.XGBRegressor(objective ='binary:hinge',colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 0.95, min_child_weight=1)

In [155]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.95, seed=None,
       silent=True, subsample=1)

In [156]:
preds = xg_reg.predict(X_test)

In [157]:
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.813849948859876
Test acuracy:  0.8153762858689767
ROC auc score:  0.7711851008663758
Confusion matrix: 
[[1441  316]
 [  25   65]]


### KNN###


In [70]:

knn = KNeighborsClassifier(n_neighbors=2)

# FIT (this does nothing in normal KNN)
knn.fit(X_train, y_train)

# predict the response 
preds = knn.predict(X_test)

train_accuracy = accuracy_score(y_train, knn.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)


Train accuracy:  0.9535527344925094
Test acuracy:  0.9491066594477531
ROC auc score:  0.49886169607285147
Confusion matrix: 
[[1753    4]
 [  90    0]]


** RESULTADOS OBTENIDOS: **

XG: 0.4995862068965517

RF: 0.4975288303130148

KNN:  0.49875862068965515